## problem 2

In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn import svm
import numpy as np
import matplotlib.pyplot as plt
import math

In [ ]:
def display_SVs(kernel_type, C_opt, deg = 1, gamma_opt=0):
    if kernel_type == 'poly':
        clf = svm.SVC(C=C_opt,kernel='poly',degree=deg,coef0=1.0)
    elif kernel_type == 'rbf':
        clf = svm.SVC(C=C_opt,kernel='rbf',gamma=gamma_opt)

    clf.fit(X_train,y_train)

    SVs = clf.support_vectors_
    alpha = clf.dual_coef_

    xi = 1 - np.squeeze(np.sign(alpha))*clf.decision_function(SVs)

    origidx = clf.support_

    sortidx = np.argsort(xi)

    topSVs = SVs[sortidx[-16:],:]
    topSVlabels = y_train[origidx[sortidx[-16:]]]

    f, axarr = plt.subplots(4, 4, figsize = (10, 10))
    ii=0
    jj=0
    for idx in range(16):
        axarr[ii, jj].imshow(topSVs[idx].reshape((28,28)), cmap='gray')
        axarr[ii, jj].axis('off')
        axarr[ii, jj].set_title('{label}'.format(label=int(topSVlabels[idx])))
        ii = ii + 1
        if ii==4:
            ii = 0
            jj = jj+1

    filename = ''
    if kernel_type == 'poly':
        if deg == 1:
            filename += 'lin'
        elif deg == 2:
            filename += 'quad'
    elif kernel_type == 'rbf':
        filename += 'rbf'

    filename += 'SVs.png'

    plt.savefig(filename)
    plt.show()

In [ ]:
np.random.seed(2022)

mnist = fetch_openml("mnist_784", version=1, as_frame=False)
X = mnist.data
y = mnist.target.astype(int)

# View the jth image
# j = 1
# plt.title('The jth image is a {label}'.format(label=int(y[j])))
# plt.imshow(X[j].reshape((28,28)), cmap='gray')
# plt.show()

X4 = X[y==4,:]
X9 = X[y==9,:]
y4 = np.full(X4.shape[0], 4)
y9 = np.full(X9.shape[0], 9)

In [ ]:
# Create training/validation/testing sets for each class
_X4, X_test4, _y4, y_test4 = train_test_split(X4, y4, train_size = 4000)
X_train4, X_val4, y_train4, y_val4 = train_test_split(_X4, _y4, train_size = 2000)
_X9, X_test9, _y9, y_test9 = train_test_split(X9, y9, train_size = 4000)
X_train9, X_val9, y_train9, y_val9 = train_test_split(_X9, _y9, train_size = 2000)

# Vertical stack to have one X vector each for train/validate/test
X_train = np.vstack((X_train4, X_train9))
X_val = np.vstack((X_val4, X_val9))
X_test = np.vstack((X_test4, X_test9))

# Combine to have one y vector each for train/validate/test
y_train = np.append(y_train4, y_train9)
y_val = np.append(y_val4, y_val9)
y_test = np.append(y_test4, y_test9)

In [ ]:
Cvec = np.logspace(-3, 3, 20)

for deg in [1, 2]:
    print("Training SVM with polynomial kernel, deg = " + str(deg))
    Pe_train = []   # Probability of error for training set
    Pe_val = []     # Probability of error for validation set
    Pe_test = []    # Probability of error for testing set
    numSvs = []     # Number of support vectors

    for Cval in Cvec:
        print('Testing Cval = ' + str(Cval))
        
        # Train SVM w/ polynomial kernel using Cval and deg
        clf = svm.SVC(C = Cval, kernel = 'poly', degree = deg)
        clf.fit(X_train, y_train)
        
        # Compute the probability of error on all 3 sets and store
        Pe_train.append(1 - clf.score(X_train, y_train))
        Pe_val.append(1 - clf.score(X_val, y_val))
        Pe_test.append(1 - clf.score(X_test, y_test))

        # Track the number of support vectors
        numSvs.append(clf.support_vectors_.shape[0])

    # Based on the validation set, set the value for C
    Cval_opt = Cvec[np.argmin(Pe_val)]
    print('Optimal value of C based on validation set: ' + str(Cval_opt))
    print('Train error: ' + str(Pe_train[np.argmin(Pe_val)]))
    print('Test error: ' + str(Pe_test[np.argmin(Pe_val)]))
    print('Number of support vectors: ' + str(numSvs[np.argmin(Pe_val)]))

    # Visualize how training/validation/testing error and # of SVs changes with C
    with np.printoptions(precision = 4):
        print('Pe_train: ', np.array(Pe_train))
        print('Pe_val: ', np.array(Pe_val))
        print('Pe_test: ', np.array(Pe_test))
    print('numSvs: ', numSvs)

    # Display the SVs that are hardest to classify
    display_SVs('poly', Cval_opt, deg = deg)

In [ ]:
Pe_train = []   # Probability of error for training set
Pe_val = []     # Probability of error for validation set
Pe_test = []    # Probability of error for testing set
numSvs = []     # Number of support vectors

Gammavec = np.logspace(-9, 1, 20)

for Gammaval in Gammavec:
    print('Testing Gammaval = ' + str(Gammaval))
    
    # Train SVM w/ rbf kernel using C = 10 and Gammaval
    clf = svm.SVC(C = 10, kernel = 'rbf', gamma = Gammaval)
    clf.fit(X_train, y_train)

    # Compute the probability of error on all 3 sets and store
    Pe_train.append(1 - clf.score(X_train, y_train))
    Pe_val.append(1 - clf.score(X_val, y_val))
    Pe_test.append(1 - clf.score(X_test, y_test))

    # Track the number of support vectors
    numSvs.append(clf.support_vectors_.shape[0])

Gammaval_opt = Gammavec[np.argmin(Pe_val)]
print('Optimal value of Gamma based on validation set: ' + str(Gammaval_opt))
print('Train error: ' + str(Pe_train[np.argmin(Pe_val)]))
print('Test error: ' + str(Pe_test[np.argmin(Pe_val)]))
print('Number of support vectors: ' + str(numSvs[np.argmin(Pe_val)]))

with np.printoptions(precision = 4):
    print('Pe_train: ', np.array(Pe_train))
    print('Pe_val: ', np.array(Pe_val))
    print('Pe_test: ', np.array(Pe_test))
print('numSvs: ', numSvs)

display_SVs('rbf', 10, gamma_opt = Gammaval_opt)

## problem 3

In [ ]:
import numpy as np
from sklearn.svm import SVR
import matplotlib.pyplot as plt
from math import exp

In [ ]:
# Generate training and testing datasets
np.random.seed(2022)

n = 100

X_train = np.random.rand(n)
y_train = np.sin(9 * X_train) + np.sqrt(1 / 3.0) * np.random.randn(n)

X_test = np.linspace(0, 1, 100)
y_test = np.sin(9 * X_test)

In [ ]:
# krr class for simplicity
class krr:    
    def __init__(self, _X_train, _y_train):
        self.X_train = _X_train
        self.y_train = _y_train.reshape(-1, 1)
        
    def predict(self, l, g, X_test, y_test):
        n = self.X_train.shape[0]
        
        # Generate K matrix
        K = np.empty((n, n))
        for i in range(n):
            for j in range(n):
                # radial basis function
                K[i, j] = exp(-g * (self.X_train[i] - self.X_train[j]) ** 2)
        
        # Predict y for each X sample in X_test
        y_pred = np.empty(n)
        for i in range(n):
            k_x = np.empty(n)
            for j in range(n):
                # radial basis function
                k_x[j] = exp(-g * (self.X_train[j] - X_test[i]) ** 2)
            y_pred[i] = self.y_train.T @ np.linalg.inv(K + l * np.identity(n)) @ k_x.reshape(1, -1).T
        
        # For optimization of parameters calculate MSE
        mse = (np.square(y_pred - y_test)).mean()

        return y_pred, mse

In [ ]:
# kernelized ridge regression with radial basis function
Pe_test = []

Lvec = np.logspace(-3, 3, 20)
Gvec = np.logspace(-3, 3, 20)

reg = krr(X_train, y_train)

# Predict y values for each value of gamma
for g in Gvec:
    y_pred, mse = reg.predict(1.0, g, X_test, y_test)
    Pe_test.append(mse)

Gopt = Gvec[np.argmin(Pe_test)]
print('Optimal gamma value: ', Gopt)

Pe_test = []

# Predict y values for each value of lambda, given optimal gamma
for l in Lvec:
    y_pred, mse = reg.predict(l, Gopt, X_test, y_test)
    Pe_test.append(mse)

Lopt = Lvec[np.argmin(Pe_test)]
print('Optimal lambda value: ', Lopt)

# Get predicted y values and MSE for optimized parameters
y_pred, mse = reg.predict(Lopt, Gopt, X_test, y_test)

print('Test MSE: ', mse)

plt.scatter(X_test, y_pred, label = 'Predicted');
plt.scatter(X_test, y_test, label = 'Actual');
plt.legend();
plt.xlabel('X');
plt.ylabel('Y');

In [ ]:
# kernelized support vector regression with radial basis function
Pe_test = []

Cvec = np.logspace(-3, 3, 20)
Gvec = np.logspace(-2, 2, 20)
Evec = np.logspace(-6, -1, 20)

# Fit and predict SVR for each cost value
for C in Cvec:
    reg = SVR(C = C, epsilon = 0.1, kernel = 'rbf', gamma = 5.0)
    reg.fit(X_train.reshape(-1, 1), y_train)
    Pe_test.append(1 - reg.score(X_test.reshape(-1, 1), y_test.reshape(-1, 1)))

Copt = Cvec[np.argmin(Pe_test)]
print('Optimal cost value: ', Copt)

Pe_test = []

# Fit and predict SVR for each gamma value, given optimal cost
for g in Gvec:
    reg = SVR(C = Copt, epsilon = 0.1, kernel = 'rbf', gamma = g)
    reg.fit(X_train.reshape(-1, 1), y_train)
    Pe_test.append(1 - reg.score(X_test.reshape(-1, 1), y_test.reshape(-1, 1)))

Gopt = Gvec[np.argmin(Pe_test)]
print('Optimal gamma value: ',  Gopt)

Pe_test = []

# Fit and predict SVR for each epsilon value, given optimal cost and gamma
for e in Evec:
    reg = SVR(C = Copt, epsilon = e, kernel = 'rbf', gamma = Gopt)
    reg.fit(X_train.reshape(-1, 1), y_train)
    Pe_test.append(1 - reg.score(X_test.reshape(-1, 1), y_test.reshape(-1, 1)))

Eopt = Evec[np.argmin(Pe_test)]
print('Optimal epsilon value: ', Eopt)

# Fit and predict SVR using all three optimized parameters
reg = SVR(C = Copt, epsilon = Eopt, kernel = 'rbf', gamma = Gopt)
reg.fit(X_train.reshape(-1, 1), y_train)
y_pred = reg.predict(X_test.reshape(-1, 1))

print('Test error: ', 1 - reg.score(X_test.reshape(-1, 1), y_test.reshape(-1, 1)))

plt.scatter(X_test, y_pred, label = 'Predicted');
plt.scatter(X_test, y_test, label = 'Actual');
plt.legend();
plt.xlabel('X');
plt.ylabel('Y');